In [1]:
import csv
import re
import string

import pandas as pd

In [2]:
data = pd.read_csv(
    r"/Users/richmondsin/Desktop/DSA4264/DSA4264-Detoxify/kedro-data/data/03_primary/clean_gt_texts_data.csv/2024-10-24T05.15.04.503Z/clean_gt_texts_data.csv"
)
print(data.shape)

(4381725, 10)


In [3]:
sample_texts = data["text"].sample(n=10, random_state=42)  # Get random samples
print(sample_texts)

868364     Ngl, I'd be worried. Who gives out phone numbe...
1490409    Circuit breaker is also a widely used word in ...
2921161    Private out lol. Gov job nowadays also all giv...
2674987    I studied in New Zealand for 8 years amp was t...
4056624    Singapores steepest drop in birth rate was fro...
4080688    Heres ten cents, my two cents are free Guess a...
2559174    is it really ur leave day if u aren't getting ...
1319148              Can definitely sleep well at night nao!
3099057                      They claim Bukit Batok province
3691036    IT'S IKEA'S STRANDMON! i'm sorry idk why i sai...
Name: text, dtype: object


In [4]:
print(data.columns.tolist())

['text', 'timestamp', 'username', 'link', 'link_id', 'parent_id', 'id', 'subreddit_id', 'moderation', 'year']


In [5]:
# Calculate the word count for each text entry
data["word_count"] = data["text"].str.split(" ").str.len()

# Count the occurrences of each unique word count
word_count_counts = data["word_count"].value_counts().sort_index()

# Display the number of unique word counts
num_unique_word_counts = word_count_counts.count()
print(f"Number of unique word counts: {num_unique_word_counts}")

# Display more results
word_count_counts_display = word_count_counts[word_count_counts > 0]
print(
    word_count_counts_display.head(100)
)  # Display the first 100 word counts for a quick overview
print(
    word_count_counts_display.tail(100)
)  # Display the last 100 word counts for completeness

Number of unique word counts: 1641
word_count
1.0      126673
2.0      130028
3.0      152811
4.0      166991
5.0      173137
          ...  
96.0       4571
97.0       4332
98.0       4195
99.0       4029
100.0      4096
Name: count, Length: 100, dtype: int64
word_count
1589.0    2
1590.0    1
1591.0    2
1592.0    2
1593.0    1
         ..
1840.0    1
1842.0    1
1864.0    1
1868.0    1
1872.0    1
Name: count, Length: 100, dtype: int64


In [6]:
# Calculate mean and median word count
mean_word_count = data["word_count"].mean()
median_word_count = data["word_count"].median()

print(f"Mean word count: {mean_word_count:.2f}")
print(f"Median word count: {median_word_count:.2f}")

Mean word count: 32.32
Median word count: 16.00


In [7]:
# Count entries with more than 5 words and 80 words or less
count_entries = data[(data["word_count"] >= 5) & (data["word_count"] <= 50)].shape[0]

print(f"Number of entries with more than 5 words and 50 words or less: {count_entries}")

Number of entries with more than 5 words and 50 words or less: 3117245


In [8]:
# one_char_texts = data[data['text'].str.len() == 5]
five_words = data[data["word_count"] == 5]
# Get 10 examples
examples = (
    five_words.sample(n=10, random_state=62) if len(five_words) >= 10 else five_words
)

# Display the examples
print(examples[["text"]])

                                           text
1041425               You broke up recently too
2341273              How to reschedule Help pls
2075990                         Why M and not P
2413377                 Knn 14 minimum is crazy
642730                 I smell a lawsuit coming
1652877             Yeah that's the plus point!
3395226          Thank you for the explanation.
3030423                Or cher in the classroom
2667886             Eating in moderation, is ok
3866161  aka dishonorable discharge. SAY SORRY!


In [9]:
# Shuffle whole dataset
shuffled_data = data.sample(frac=1, random_state=42).reset_index(drop=True)

In [10]:
# Filter for entries with at least 5 words and less than or equal to 50 words
filtered_data = shuffled_data[
    (shuffled_data["word_count"] >= 5) & (shuffled_data["word_count"] <= 50)
]

# Save the filtered out data (those not meeting the word count criteria)
filtered_out_data = shuffled_data[~shuffled_data.index.isin(filtered_data.index)]

# Take a subsample of 1 million rows
subset_size = min(1000000, len(filtered_data))
subset = filtered_data.iloc[:subset_size]

# Save the remaining non-filtered data
remaining_data = filtered_data.iloc[subset_size:]

# Combine filtered out data and remaining data
combined_data = pd.concat([filtered_out_data, remaining_data], ignore_index=True)

# Save the combined data to a CSV file
combined_data.to_csv("other_than_1million.csv", index=False)

# Split the subset into 5 different CSV files
num_files = 5
rows_per_file = len(subset) // num_files

for i in range(num_files):
    start_idx = i * rows_per_file
    end_idx = (
        (i + 1) * rows_per_file if i < num_files - 1 else len(subset)
    )  # Handle the last file
    subset.iloc[start_idx:end_idx].to_csv(
        f"1million_subset_part_{i + 1}.csv",
        index=False,
    )

print(
    "Subsampling complete. Files saved as 'text_subset_part_1.csv' to 'text_subset_part_5.csv' and 'other_than_1million.csv'."
)

Subsampling complete. Files saved as 'text_subset_part_1.csv' to 'text_subset_part_5.csv' and 'other_than_1million.csv'.


In [11]:
print(combined_data.shape)

(3381725, 11)
